In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from walks import Route, Point, Segment

In [16]:
from collections import defaultdict
import statistics

In [17]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [19]:
from db_routes import load_routes_from_db

In [20]:
requested_map = cimgt.OSM()  # Open Street Map

In [21]:
routes = load_routes_from_db()

for route_id in range(len(routes) - 5, len(routes) + 1):
    rt = routes[route_id]
    print(f'Route {route_id}: {len(rt.points)} points, distance: {rt.metres() / 1000:.3f}km = {rt.miles():.2f}mi, {rt.time():.2f} sec = {rt.avg_mph():.2f} mph')    


57563
(0.054034, 51.377139, 0.184897, 51.462176)
Route 70: 1105 points, distance: 7.408km = 4.60mi, 4501.88 sec = 3.68 mph
Route 71: 950 points, distance: 6.200km = 3.85mi, 3680.46 sec = 3.77 mph
Route 72: 1444 points, distance: 9.774km = 6.07mi, 6059.22 sec = 3.61 mph
Route 73: 1118 points, distance: 7.413km = 4.61mi, 4338.00 sec = 3.82 mph
Route 74: 1222 points, distance: 8.746km = 5.43mi, 5294.27 sec = 3.70 mph
Route 75: 1019 points, distance: 7.209km = 4.48mi, 4334.18 sec = 3.72 mph


In [ ]:
def locs_at(mins):
    pts = [rt.point_at(mins * 60) for rt in routes.values()]

    lons = [pt.lon for pt in pts if pt]
    lats = [pt.lat for pt in pts if pt]

    return (lons, lats)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(projection=requested_map.crs))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

ax.add_image(requested_map, 14)

lons, lats = locs_at(15)
plt.scatter(lons, lats, color='aqua', s=100, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(30)
plt.scatter(lons, lats, color='indigo', s=100, alpha=1, transform=ccrs.PlateCarree())

lons, lats = locs_at(45)
plt.scatter(lons, lats, color='red', s=100, alpha=1, transform=ccrs.PlateCarree())

avg_start_lon = statistics.mean(rt.points[0].lon for rt in routes.values())
avg_start_lat = statistics.mean(rt.points[0].lat for rt in routes.values())
plt.scatter([avg_start_lon], [avg_start_lat], color='lime', s=1000, alpha=1, marker='o', transform=ccrs.PlateCarree())

plt.title(f'After 15, 30, 45 minutes')

plt.show()

In [ ]:
lons = [p.lon for p in points]
lats = [p.lat for p in points]
eles = [p.ele for p in points]
min_ele = min(eles)
max_ele = max(eles)

In [ ]:
def col_for(e, min_e, max_e):
    index = (e - min_e) / (max_e - min_e)
    return (index, index, index)
    
cols = [col_for(e, min_ele, max_ele) for e in eles]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20), subplot_kw=dict(projection=requested_map.crs))

gl = ax.gridlines(draw_labels=True)
gl.top_labels = gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = route_bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

ax.set_extent(extent)

ax.add_image(requested_map, 14)

plt.scatter(lons, lats, color=cols, s=50, alpha=1, transform=ccrs.PlateCarree())

plt.title(f'Elevations')

plt.show()

In [ ]:
print(len(lons), len(lats), len(cols), len(points))